Questo notebook utilizza un modello CUT pre-addestrato per trasformare le immagini del dominio **Reale** nello stile del dominio **Sintetico**.

**Prerequisiti:**
1. Avere il Dataset delle immagini caricato (es. `EGO-CH-OBJ-SEG`).
2. Avere il Dataset dei pesi caricato (il file `.pth` ottenuto dal training).

In [ ]:
import os
import shutil


%cd /kaggle/working
if os.path.exists("benchmarking-generative-models-for-domain-adaptation"):
    shutil.rmtree("benchmarking-generative-models-for-domain-adaptation")


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd benchmarking-generative-models-for-domain-adaptation


print("Installo librerie...")
!pip install -q dominate visdom



In [ ]:
# Patch per il bug noto di argparse in cut_model.py
target_file = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/cut/models/cut_model.py"

if os.path.exists(target_file):
    with open(target_file, "r") as f:
        content = f.read()
    
    new_content = content.replace("choices='(CUT, cut, FastCUT, fastcut)'", "choices=['CUT', 'cut', 'FastCUT', 'fastcut']")
    
    with open(target_file, "w") as f:
        f.write(new_content)
    print("Patch applicata correttamente.")
else:
    print("Errore: File cut_model.py non trovato.")

In [ ]:
import os
import shutil
import glob

# --- CONFIGURAZIONE PERCORSI ---
#NOTA: i percorsi vanno personalizzati
KAGGLE_REAL_IMAGES = "/kaggle/input/ego-ch/EGO-CH-OBJ-SEG/real/train/frames"
REPO_DIR = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation"
CUT_DIR = os.path.join(REPO_DIR, "models/cut")
TEST_A_DIR = os.path.join(CUT_DIR, "datasets/ego_adaptation/testA")

# --- PREPARAZIONE DATASET COMPLETO ---
%cd {CUT_DIR}

if os.path.exists(TEST_A_DIR):
    shutil.rmtree(TEST_A_DIR)
os.makedirs(TEST_A_DIR, exist_ok=True)

# Copia fisica delle immagini
print(f"Copia di tutte le immagini da {KAGGLE_REAL_IMAGES}...")
image_list = glob.glob(os.path.join(KAGGLE_REAL_IMAGES, "*.*"))

for img_path in image_list:
    shutil.copy(img_path, TEST_A_DIR)

num_img = len(os.listdir(TEST_A_DIR))
print(f"Copiate {num_img} immagini. Pronto per la generazione.")

In [ ]:
print("Avvio Generazione (Domain Adaptation)...")

!python test.py \
  --dataroot ./datasets/ego_adaptation \
  --name ego_cut_kaggle \
  --CUT_mode CUT \
  --phase test \
  --epoch 40 \
  --num_test {num_img}

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import glob

# Percorsi risultati
real_path = "./results/ego_cut_kaggle/test_40/images/real_A"
fake_path = "./results/ego_cut_kaggle/test_40/images/fake_B"

# Trova immagini
images = glob.glob(os.path.join(real_path, "*.png")) + glob.glob(os.path.join(real_path, "*.jpg"))

if len(images) > 0:
    samples = random.sample(images, min(3, len(images)))
    
    plt.figure(figsize=(12, 8))
    for i, img_path in enumerate(samples):
        filename = os.path.basename(img_path)
        
        # Carica Reale e Fake
        img_real = Image.open(os.path.join(real_path, filename))
        img_fake = Image.open(os.path.join(fake_path, filename))
        
        # Mostra
        plt.subplot(3, 2, 2*i + 1)
        plt.imshow(img_real)
        plt.title("Input Reale")
        plt.axis('off')
        
        plt.subplot(3, 2, 2*i + 2)
        plt.imshow(img_fake)
        plt.title("Output Sintetico (CUT)")
        plt.axis('off')
    plt.show()
else:
    print("Nessuna immagine trovata nei risultati.")

In [ ]:
print("Generazione completata. Creazione dello ZIP finale...")
RESULT_DIR = "./results/ego_cut_kaggle/test_40/images/fake_B/"
shutil.make_archive("/kaggle/working/DATASET_DOMA_ADAPTED", 'zip', RESULT_DIR)

print("\nZip pronto.")
print("Le immagini trasformate sono in: " + RESULT_DIR)